<a href="https://colab.research.google.com/github/ArianeMora/enzyme-tk/blob/main/Enzymetk_Reactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

! conda install -c conda-forge -c bioconda foldseek -y
! conda install -c conda-forge -c bioconda mmseqs2 -y

! pip install enzymetk
! pip install unimol_tools
! pip install rdkit

✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - foldseek


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aria2-1.37.0               |       hbc8128a_2         1.6 MB  conda-forge
    certifi-2026.1.4           |     pyhd8ed1ab_0         147 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    foldseek-10.941cd33        |       h5021889_1       410.2 MB  bioconda
    gawk-5.3.1                 |       hcd3d067_0         1.1 MB  conda-forge
    gmp-6.3.0                  |       hac33072_2         449 KB  conda-forge
    libasprintf-0.25.1         |       h3f43e3d_1          52 KB  conda-forge
    libgettextpo-0.25.1        |       h5888daf_0         174 KB  conda-forge
    mpfr-4.2.1   

In [49]:
from google.colab import drive
import pandas as pd
import torch

drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/shared_resources/enzymetk"

# read in reaction data
reaction_df = pd.read_csv(f'{DATA_DIR}/enzymemap_v2_brenda2023.csv')
# read in enzyme data
enzyme_df = pd.read_csv(f'{DATA_DIR}/uniprotkb_reviewed_true_2025_17_02.tsv', sep='\t')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Example workflow to accompany Enzyme-tk

This is an example workflow for starting with a reaction of interest. A researcher may be curious which of the enzymes annotated in Brenda/EnzymeMap/UniProt may have the same reaction or act on a similar substrate.


This notebook seeks to run through that simple pipeline using enzyme-tk and the datasets provided by it.

# Datasets

## UniProt

Data were downloaded from UniProt on the 17th of February 2025 by selecting reviewed sequences and the following additional information (Fragment, Length, Mass, Organism ID, Taxonomic Lineage (IDs), Active Site, Cofactor, EC number, Reviewed), totaling 572,970 protein sequences. These were filtered for size (minimum length 100, maximum 1024), to be non-fragments, and proteins with an EC annotation, resulting in 261,254 potential enzymes.

## Reactions
Reaction data were downloaded from EnzymeMap,59 containing 62,896 deduplicated reactions, or 54,090 products and 37,263 substrates. Properties were calculated for each substrate and product by using RdKit version 2024.3.3, calculated properties were molecular weight (MolWt), total polar surface area (TPSA), calculated solubility proxy (MolLogP), and the maximum and minimum Gasteiger partial charges (MaxPartialCharge and MinPartialCharge).



In [26]:
from __future__ import annotations
import pandas as pd
from sciutil import SciUtil
import timeit
import logging
import subprocess
import os
import typer
from pathlib import Path
import random
import string

u = SciUtil()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

def run_script(script_name: str, verbose: bool = True) -> int:
    """Run a shell script from the conda_envs directory."""
    script_path = Path(f'{SCRIPT_DIR}/{script_name}')

    if not script_path.exists():
        typer.secho(f"Error: Script {script_name} not found at {script_path}", fg=typer.colors.RED)
        raise typer.Exit(1)

    typer.secho(f"Running {script_name}...", fg=typer.colors.BLUE)

    try:
        result = subprocess.run(
            ["bash", str(script_path)],
            cwd=str(SCRIPT_DIR),
            check=True,
            capture_output=not verbose,
            text=True,
        )
        typer.secho(f"✓ {script_name} completed successfully", fg=typer.colors.GREEN)
        return 0
    except subprocess.CalledProcessError as e:
        typer.secho(f"✗ {script_name} failed with exit code {e.returncode}", fg=typer.colors.RED)
        if not verbose and e.stderr:
            typer.echo(e.stderr)
        raise typer.Exit(1)

class Pipeline():

    def __init__(self, *steps: Step):
        self.steps = list(steps)

    def __rshift__(self, other: Step) -> Step:
        return Pipeline(*self.steps, other)

    def execute(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Execute some shit.
        """
        for step in self.steps:
            df = step.execute(df)
        return df

    def __rlshift__(self, other: pd.DataFrame) -> pd.DataFrame:
        return self.execute(other)


class Step():
    def __init__(self):
        # Should only have one of these
        self.venv = None
        self.conda = None
        self.exec = "/bin/bash"


    def execute(self, df: pd.DataFrame) -> pd.DataFrame:
        """ Execute some shit """
        return df

    def install_venv(self, env_args=None):
        self.conda = None
        self.venv = None
        """ Install unimol_tools """
        cmd = ['uv', 'venv', self.env_name]
        if env_args:
            cmd.extend(env_args)
        self.run(cmd)
        # Ensure pip is up to date and installed
        try:
          cmd = [f'{self.env_name}/bin/python', 'pip', 'install', '--upgrade', 'pip']
          self.run(cmd)
        except:
          # Need to have this for jupyter envs
          cmd = ['wget', 'https://bootstrap.pypa.io/get-pip.py']
          self.run(cmd)
          cmd = [f'{self.env_name}/bin/python', 'get-pip.py']
          self.run(cmd)


    def install_conda(self, env_args=None):
        return

    def run(self, cmd: list):
        """ Run a command """
        result = None
        start = timeit.default_timer()
        # Prioitize running in a venv if we have it
        if self.venv:
            cmd = [self.venv] + cmd
            u.warn_p(['Running in venv:', self.venv])
        elif self.conda:
            cmd = ['conda', 'run', '-n', self.conda] + cmd
        u.dp(['Running command', ' '.join([str(c) for c in cmd])])

        result = subprocess.run(cmd, capture_output=True,
                                text=True,
                                check=True)

        u.warn_p(['Output:'])
        print(result.stdout)
        if result.stderr:
            u.err_p(['Error:', result.stderr])
            logger.error(result.stderr)
        logger.info(result.stdout)
        u.dp(['Time for command to run (min): ', (timeit.default_timer() - start)/60])
        return result

    def __rshift__(self, other: Step)   :
        return Pipeline(self, other)

    def __rlshift__(self, other: pd.DataFrame) -> pd.DataFrame:
        """
        Overriding the right shift operator to allow for the pipeline to be executed.
        """
        return self.execute(other)

import pandas as pd
import numpy as np
from tempfile import TemporaryDirectory
import pandas as pd
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool

try:
    from rdkit import Chem
    from rdkit import DataStructs
    from rdkit.Chem import rdChemReactions
    from rdkit.DataStructs import FingerprintSimilarity
    from rdkit.Chem.Fingerprints import FingerprintMols

except ImportError as e:
    print("ReactionDist: Needs rdkit package. Install with pip install rdkit.")



class ReactionDist(Step):

    def __init__(self, id_column_name: str, smiles_column_name: str, smiles_string: str, num_threads=1):
        self.smiles_column_name = smiles_column_name
        self.id_column_name = id_column_name
        self.smiles_string = smiles_string
        self.num_threads = num_threads

    def __execute(self, data: list) -> np.array:
        reaction_df = data
        rows = []
        fp_params = rdChemReactions.ReactionFingerprintParams()
        rxn = rdChemReactions.ReactionFromSmarts(self.smiles_string)
        rxn_fp = rdChemReactions.CreateStructuralFingerprintForReaction(rxn, ReactionFingerPrintParams=fp_params) #rdChemReactions.CreateStructuralFingerprintForReaction(rxn, ReactionFingerPrintParams=fp_params)

        # compare all fp pairwise without duplicates
        for smile_id, smiles in tqdm(reaction_df[[self.id_column_name, self.smiles_column_name]].values): # -1 so the last fp will not be used
            mol_ = rdChemReactions.ReactionFromSmarts(smiles)
            # Note: if you don't pass , ReactionFingerPrintParams=fp_params you get different results
            # i.e. reactions that don't appear to be the same are reported as similar of 1.0
            # https://github.com/rdkit/rdkit/discussions/5263
            fps = rdChemReactions.CreateStructuralFingerprintForReaction(mol_, ReactionFingerPrintParams=fp_params)
            rows.append([smile_id,
                         self.smiles_string,
                         smiles,
                         DataStructs.TanimotoSimilarity(fps, rxn_fp),
                         DataStructs.RusselSimilarity(fps, rxn_fp),
                         DataStructs.CosineSimilarity(fps, rxn_fp)])
        distance_df = pd.DataFrame(rows, columns=[self.id_column_name, 'QuerySmiles', self.smiles_column_name, 'TanimotoSimilarity', 'RusselSimilarity', 'CosineSimilarity'])
        return distance_df

    def execute(self, df: pd.DataFrame) -> pd.DataFrame:
        if self.num_threads > 1:
            data = []
            df_list = np.array_split(df, self.num_threads)
            for df_chunk in df_list:
                data.append(df_chunk)
            pool = ThreadPool(self.num_threads)
            output_filenames = pool.map(self.__execute, data)
            df = pd.DataFrame()
            for tmp_df in output_filenames:
                df = pd.concat([df, tmp_df])
            return df

        else:
            return self.__execute(df)



try:
    from rdkit import Chem
    from rdkit import DataStructs
    from rdkit.Chem import rdChemReactions
    from rdkit.DataStructs import FingerprintSimilarity
    from rdkit.Chem import rdFingerprintGenerator
    from rdkit.Chem.Fingerprints import FingerprintMols
except ImportError as e:
    print("SubstrateDist: Needs rdkit package. Install with pip install rdkit.")


class SubstrateDist(Step):

    def __init__(self, id_column_name: str, smiles_column_name: str, smiles_string: str, num_threads=1):
        self.smiles_column_name = smiles_column_name
        self.id_column_name = id_column_name
        self.smiles_string = smiles_string
        self.num_threads = num_threads

    def __execute(self, data: list) -> np.array:
        reaction_df = data
        tmp_label = ''.join(random.choices(string.ascii_letters + string.digits, k=10))

        rxn = Chem.MolFromSmiles(self.smiles_string)
        # Switched to using morgan fingerprints https://greglandrum.github.io/rdkit-blog/posts/2023-01-18-fingerprint-generator-tutorial.html
        # followed this tutorial
        mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=2048)
        rxn_fp = mfpgen.GetFingerprint(rxn)
        rows = []
        # compare all fp pairwise without duplicates
        for smile_id, smiles in tqdm(reaction_df[[self.id_column_name, self.smiles_column_name]].values): # -1 so the last fp will not be used
            mol_ = Chem.MolFromSmiles(smiles)
            fps = mfpgen.GetFingerprint(mol_)
            rows.append([smile_id,
                         self.smiles_string,
                         smiles,
                         DataStructs.TanimotoSimilarity(fps, rxn_fp),
                         DataStructs.RusselSimilarity(fps, rxn_fp),
                         DataStructs.CosineSimilarity(fps, rxn_fp)])
        distance_df = pd.DataFrame(rows, columns=[self.id_column_name, 'QuerySmiles', self.smiles_column_name, 'TanimotoSimilarity', 'RusselSimilarity', 'CosineSimilarity'])
        return distance_df

    def execute(self, df: pd.DataFrame) -> pd.DataFrame:
        if self.num_threads > 1:
            data = []
            df_list = np.array_split(df, self.num_threads)
            for df_chunk in df_list:
                data.append(df_chunk)
            pool = ThreadPool(self.num_threads)
            output_filenames = pool.map(self.__execute, data)
            df = pd.DataFrame()
            for tmp_df in output_filenames:
                df = pd.concat([df, tmp_df])
            return df

        else:
            return self.__execute(df)


class UniMol(Step):

    def __init__(self, smiles_col: str, unimol_model = 'unimolv2',
                 unimol_size = '164m', num_threads = 1,
                 env_name = 'enzymetk', venv_name = None):
        super().__init__()
        self.smiles_col = smiles_col
        self.num_threads = num_threads
        self.conda = env_name
        self.env_name = env_name
        self.venv = venv_name if venv_name else f'{env_name}/bin/python'
        self.unimol_model = unimol_model
        self.unimol_size = unimol_size


    def install(self, env_args=None):
        # e.g. env args could by python=='3.1.1.
        self.install_venv(env_args)
        # Now the specific
        try:
            cmd = [f'{self.env_name}/bin/pip', 'install', 'unimol_tools']
            self.run(cmd)
        except Exception as e:
            cmd = [f'{self.env_name}/bin/pip3', 'install', 'unimol_tools']
            self.run(cmd)
        self.run(cmd)
        # Now set the venv to be the location:
        self.venv = f'{self.env_name}/bin/python'

    def __execute(self, df: pd.DataFrame) -> pd.DataFrame:
        smiles_list = list(df[self.smiles_col].values)
        reprs = []
        for smile in smiles_list:
            try:
                unimol_repr = self.clf.get_repr([smile], return_atomic_reprs=True)
                reprs.append(unimol_repr['cls_repr'])
            except Exception as e:
                logger.warning(f"Error embedding smile {smile}: {e}")
                reprs.append(None)
        df['unimol_repr']  = reprs
        return df

    def execute(self, df: pd.DataFrame) -> pd.DataFrame:
        try:
            from unimol_tools import UniMolRepr
        except ImportError as e:
            raise ImportError(
                "UniMolRepr requires unimol-tools. "
                "Install after initializing class with install()"
            ) from e
        # single smiles unimol representation
        clf = UniMolRepr(data_type='molecule',
                        remove_hs=False,
                        model_name= self.unimol_model or 'unimolv2', # avaliable: unimolv1, unimolv2
                        model_size= self.unimol_size or '164m', # work when model_name is unimolv2. avaliable: 84m, 164m, 310m, 570m, 1.1B.
                        )
        self.clf = clf
        with TemporaryDirectory() as tmp_dir:
            if self.num_threads > 1:
                data = []
                df_list = np.array_split(df, self.num_threads)
                for df_chunk in df_list:
                    data.append(df_chunk)
                pool = ThreadPool(self.num_threads)
                output_filenames = pool.map(self.__execute, data)
                df = pd.DataFrame()
                for tmp_df in output_filenames:
                    df = pd.concat([df, tmp_df])
                return df

            else:
                return self.__execute(df)


In [54]:
reaction_df = reaction_df.drop_duplicates('mapped')
reaction_df

,rxn_idx,mapped,unmapped,orig_rxn_text,rule,rule_id,source,steps,quality,natural,organism,protein_refs,protein_db,ec_num
0,0,[CH3:1][CH:2]=[O:3].[H+].[NH2:4][C:5](=[O:6])[...,CC=O.NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O...,acetaldehyde + NADH + H+ = ethanol + NAD+ {r},[#6:1]1=[#6:2]-[#7:3]-[#6:4]=[#6:5]-[#6:6]-1.[...,0,direct,single,0.991708,True,Saccharomyces cerevisiae,[],NaN,1.1.1.1
1,0,[CH3:1][CH2:2][OH:3].[NH2:4][C:5](=[O:6])[c:7]...,CCO.NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP...,acetaldehyde + NADH + H+ = ethanol + NAD+ {r},[#6:1]1=[#6:2]-[#7:3]-[#6:4]=[#6:5]-[#6:6]-1.[...,0,direct reversed,single,0.991708,True,Saccharomyces cerevisiae,[],NaN,1.1.1.1
25,15,[CH3:1][C:2](=[O:3])[CH:4]=[O:5].[H+].[NH2:6][...,CC(=O)C=O.NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)...,methylglyoxal + NADH + H+ = acetol + NAD+ {r},[#6:1]1=[#6:2]-[#7:3]-[#6:4]=[#6:5]-[#6:6]-1.[...,0,direct,single,0.991708,True,Candida albicans,['A0A1D8PP43'],uniprot,1.1.1.1
26,15,[CH3:1][C:2](=[O:3])[CH2:4][OH:5].[NH2:6][C:7]...,CC(=O)CO.NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)...,methylglyoxal + NADH + H+ = acetol + NAD+ {r},[#6:1]1=[#6:2]-[#7:3]-[#6:4]=[#6:5]-[#6:6]-1.[...,0,direct reversed,single,0.991708,True,Candida albicans,['A0A1D8PP43'],uniprot,1.1.1.1
27,16,[NH2:1][C:2](=[O:3])[c:4]1[cH:5][cH:6][cH:7][n...,NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)...,2-dehydro-3-deoxy-D-gluconate + NADH + H+ = 4-...,[#6:1]-[#8:2].[#6:3]1:[#6:4]:[#6:5]:[#6:6]:[#7...,1,direct,single,0.991708,True,Sphingomonas sp. A1,['A0A075B5H4'],uniprot,1.1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349447,324551,[NH2:33][c:34]1[n:35][cH:36][n:37][c:38]2[c:39...,N=c1[nH]c(=O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(...,"GTP + ATP = P1-(5'-adenosyl),P4-(5'-guanosyl) ...",[#8:1].[#8:2]-[#15:3]>>[#8:2].[#8:1]-[#15:3],281,direct,single,0.583333,False,Escherichia virus T4,[],NaN,6.5.1.3
349448,324552,[NH2:1][c:2]1[n:3][cH:4][n:5][c:6]2[c:7]1[n:8]...,Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)OP(=O)(O...,"ATPgammaS + ATP = P1,P4-bis(5'-adenosyl) tetra...",[#8:1].[#8:2]-[#15:3]>>[#8:2].[#8:1]-[#15:3],281,direct,single,0.583333,False,Escherichia virus T4,[],NaN,6.5.1.3
349449,324553,[NH2:1][c:2]1[cH:3][cH:4][n:5]([C@@H:6]2[O:7][...,Nc1ccn([C@@H]2O[C@H](COP(=O)(O)O[C@H]3[C@@H](O...,"5'-adenylated cytidine 5',3'-bisphosphate + di...",[#8:1].[#8:2]-[#15:3]>>[#8:2].[#8:1]-[#15:3],281,suggested,single,0.583333,False,Escherichia virus T4,[],NaN,6.5.1.3
349450,324553,[NH2:56][c:57]1[cH:58][cH:59][n:60]([C@@H:61]2...,N=c1[nH]c(=O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(...,"5'-adenylated cytidine 5',3'-bisphosphate + di...",[#8:1].[#8:2]-[#15:3]>>[#8:2].[#8:1]-[#15:3],281,suggested,single,0.583333,False,Escherichia virus T4,[],NaN,6.5.1.3


In [ ]:
%%capture

from enzymetk.save_step import Save
import pandas as pd
import os

rxn_smiles = 'CCCCC(COC(C1=CC=CC=C1C(OCC(CCCC)CC)=O)=O)CC>>O=C(O)C1=CC=CC=C1C(O)=O'

num_threads = 1
id_col = 'rxn_idx'
reaction_col = 'mapped'

# This takes about an hour on the 47k reactions
subset_of_reaction_df = reaction_df.sample(100)
# Do on the whole set if you need it!
# First calculate the reaction distance and then calculate the embeddings df
df = (subset_of_reaction_df << (ReactionDist(id_col, reaction_col, rxn_smiles) >> ChemBERT(id_col, reaction_col, num_threads) >> Save(f'reaction_similarity_chemberta_embeddings.pkl')))
df

DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /seyonec/PubChem10M_SMILES_BPE_450k/resolve/main/config.json HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /api/resolve-cache/models/seyonec/PubChem10M_SMILES_BPE_450k/c18fccd09b3326bf2d4633412c256d7db872156d/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /seyonec/PubChem10M_SMILES_BPE_450k/resolve/main/model.safetensors HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/seyonec/PubChem10M_SMILES_BPE_450k HTTP/1.1" 200 1334
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/seyonec/PubChem10M_SMILES_BPE_450k/commits/main HTTP/1.1" 200 3798
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/seyonec/PubChem10M_SMILES_BPE_450k/d

In [ ]:
df.sort_values(by='TanimotoSimilarity', ascending=False)

In [41]:
! pip install pubchempy

In [ ]:
import IPython.display
from rdkit import Chem
from rdkit.Chem import Draw
#import pubchempy
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from rdkit.Chem.Draw import rdMolDraw2D, rdDepictor
import rdkit

#rxn_smiles = 'O=C(N(C)C)NC1=CC=C(Cl)C=C1'

rxn = rdChemReactions.ReactionFromSmarts(rxn_smiles)
drawer = rdMolDraw2D.MolDraw2DSVG(-1,-1)
dopts = drawer.drawOptions()
dopts.bondLineWidth = 1.0 # default is 2.0
drawer.DrawReaction(rxn)
drawer.FinishDrawing()
i = IPython.display.SVG(drawer.GetDrawingText())
display(i)

names = []
smiles_to_name = {}
ms = []

for c in set(df['Reaction'].values):
    rxn = rdChemReactions.ReactionFromSmarts(c)
    #rxn = remove_atom_mapping(rxn)
    drawer = rdMolDraw2D.MolDraw2DSVG(-1,-1)
    dopts = drawer.drawOptions()
    dopts.addAtomIndices = False  # This is the key line
    dopts.bondLineWidth = 2.0 # default is 2.0
    drawer.DrawReaction(rxn)
    drawer.FinishDrawing()
    i = IPython.display.SVG(drawer.GetDrawingText())
    display(i)

In [43]:
! conda install -c conda-forge pubchempy


Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pubchempy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libblas-3.9.0              |31_h59b9bed_openblas          16 KB  conda-forge
    libcblas-3.9.0             |31_he106b2a_openblas          16 KB  conda-forge
    libgfortran-14.2.0         |       h69a702a_2          52 KB  conda-forge
    libgfortran5-14.2.0        |       hf1ad2bd_2         1.4 MB  conda-forge
    liblapack-3.9.0            |31_h7ac8fdf_openblas          16 KB  conda-forge
    libopenblas-0.3.29         |pthreads_h94d23a6_0    

In [29]:
df

,Entry,QuerySmiles,Reaction,TanimotoSimilarity,RusselSimilarity,CosineSimilarity,chemberta
0,P0DP23,CCCCC(COC(C1=CC=CC=C1C(OCC(CCCC)CC)=O)=O)CC>>O...,CCCCC(CC)COC(=O)C1=CC=CC=C1C(=O)OCC(CC)CCCC>>O...,1.000000,0.146973,1.000000,"[1.1515423, -1.2387666, -1.157189, -0.10992633..."
1,P0DP24,CCCCC(COC(C1=CC=CC=C1C(OCC(CCCC)CC)=O)=O)CC>>O...,O=C(OC(C)C)NC1=CC=CC(Cl)=C1>>O=C(O)C1=CC=CC=C1...,0.317365,0.051758,0.518221,"[1.1515423, -1.2387666, -1.157189, -0.10992633..."


# Next consider looking at the substrate instead of the reaction

In [32]:
%%capture
smiles = 'CCCCC(COC(C1=CC=CC=C1C(OCC(CCCC)CC)=O)=O)CC'

output_dir = 'tmp/'
num_threads = 1
id_col = 'Entry'
seq_col = 'Sequence'
substrate_col = 'Substrate'
rows = [['P0DP23', 'MALWMRLLPLLALLALWGPDPAAAMALWMRLLPLLALLALWGPDPAAAMALWMRLLPLLALLALWGPDPAAA', 'O=C(O)C1=CC=CC=C1C(O)=O'],
        ['P0DP24', 'MALWMRLLPLLALLALWGPDPAAAMALWMRLLPLLALLALWGPDPAAAMALWMRLLPLLALLALWGPDPAAA', 'O=C(OC(C)C)NC1=CC=CC(Cl)=C1']]
df = pd.DataFrame(rows, columns=[id_col, seq_col, substrate_col])
df = df << (SubstrateDist(id_col, substrate_col, smiles) >> UniMol(substrate_col, num_threads=num_threads) >> Save(f'substrate_similarity.pkl'))

Streaming output truncated to the last 5000 lines.
             '$phi354.0': {('$218get_iter.6',
                            State(pc_initial=196 nstack_initial=2))},
             '$phi354.1': {('$246get_iter.6',
                            State(pc_initial=224 nstack_initial=2))},
             '$phi354.2': {('$248for_iter.3',
                            State(pc_initial=248 nstack_initial=2))},
             '$phi358.0': {('$218get_iter.6',
                            State(pc_initial=196 nstack_initial=2))},
             '$phi358.1': {('$220for_iter.2',
                            State(pc_initial=220 nstack_initial=1))},
             '$phi382.0': {('$380get_iter.6',
                            State(pc_initial=358 nstack_initial=2))},
             '$phi386.0': {('$380get_iter.6',
                            State(pc_initial=358 nstack_initial=2))},
             '$phi386.1': {('$382for_iter.2',
                            State(pc_initial=382 nstack_initial=1))},
             '$phi410

In [33]:
df

,Entry,QuerySmiles,Substrate,TanimotoSimilarity,RusselSimilarity,CosineSimilarity,unimol_repr
0,P0DP23,CCCCC(COC(C1=CC=CC=C1C(OCC(CCCC)CC)=O)=O)CC,O=C(O)C1=CC=CC=C1C(O)=O,0.277778,0.004883,0.472456,"[[-2.7145216, 15.621512, -19.193138, -1.377672..."
1,P0DP24,CCCCC(COC(C1=CC=CC=C1C(OCC(CCCC)CC)=O)=O)CC,O=C(OC(C)C)NC1=CC=CC(Cl)=C1,0.188679,0.004883,0.317500,"[[-3.7447462, 16.139095, -18.375336, -2.310012..."


KeyError: 'Reaction'